FLASK WEB SITE = DEMONSTRATE FRONT END CALLS TO THE FLASK SERVER

Run each of these cells in order to see various kinds of calls to the flask server. 
Each cell has a description of the demonstration objective.

In [ ]:
import requests
import json
from datetime import datetime, timezone

In [ ]:
# Basic example to get a string from the web server request
response = requests.get('http://localhost:5000')
print(response.text)

In [ ]:
# Demonstrate a different endpoint method of getting the string
response = requests.get('http://localhost:5000/new_message')
print(response.text)

In [ ]:
# Demonstrate simulated API to return data in JSON format (could be generated in any fashion using python)
response = requests.get("http://localhost:5000/api/data")
print(response.text)

**Note:** The flask server will require a CSRF token for authentication. This example useses a session to retrieve the flask CSRF token. Then that token is inserted into the POST calls to authenticate with the flask server.

In [ ]:
# POST JSON data to the API, which will then write it to a file, and print the result of the call
post_data = {'data': f'{datetime.now(timezone.utc).isoformat()}__Newly sent from python request__'}
headers = {'Content-Type': 'application/json'}
response = requests.post('http://localhost:5000/api/data', json=post_data, headers=headers)
print(response.text)

Similar to the above call to api/data but performed using JavaScript to simulate a browser GET to the API

**Note:** Must have the console visible to see the results.  
This also demonstrates HTML/Javascript using IPython.display

In [ ]:
from IPython.display import display, HTML, Javascript

display(HTML('''
<h1>Call Flask API from Jupyter Notebook</h1>
<button onclick="callFlaskAPI()">Call API</button>
<p id="apiResponse"></p>

<script>
    async function callFlaskAPI() {
        try {
            const response = await fetch('http://localhost:5000/api/data');
            const data = await response.json();
            console.log(data);
            document.getElementById('apiResponse').innerText = JSON.stringify(data);
        } catch (error) {
            console.error('Error:', error);
        }
    }
</script>
'''))

Simple JavaScript to capture mouse clicks and write the position to the browser console.  It will also 
post it to the post_data API, which then writes it to a JSON file.

**Note:** Must have the console visible to see the results console.log results.  This also demonstrates HTML/Javascript using IPython.display.

The below example assumes the Javascript POST is contained within the same jupyter notebook as the python examples so a python requests.get is used to retrieve the simulated flask CSRF token.  The X-CSRFToken must be included in the header even though CSRF is disabled in the server.  If not an error is likely. 

In [ ]:
from IPython.display import display, HTML, Javascript

# First retrieve the simulated CSRF token from the flask server using python and make it available to follow. 
csrf_token_json = requests.get("http://localhost:5000/get_csrf_token").text
json_response = json.loads(csrf_token_json)
csrf_token = json_response['csrf_token']

display(HTML('''
<h1>Setup mouse click position alerts</h1>

<script>    
    // Create a function to handle the click event
    async function handleClick(e) {
        var xPosition = e.clientX;    // Get the horizontal coordinate
        var yPosition = e.clientY;    // Get the vertical coordinate
    
        // Display the coordinates
        var clickData = {"mouseClickPosition": {"x": xPosition, "y": yPosition}};
        
        // Add each mouse click event to console log
        console.log(clickData);   
        
        // Fetch CSRF token
        try {
            const csrfToken = "{csrf_token}";
            
            // Now post the mouse click data to the server
            // options for the fetch function
            const options = {
              method: 'POST', 
              headers: {
                'Content-Type': 'application/json', 
                'X-CSRFToken': csrfToken  // Include CSRF token in headers
              },
              body: JSON.stringify(clickData) // stringify the JS object to JSON
            };

            // Sending a POST request to the Flask server
            const response = await fetch('http://localhost:5000/api/data', options);
            if (!response.ok) {
              throw new Error(`HTTP error! status: ${response.status}`);
            }
            const json = await response.json();
            console.log(json); // log the output from the Flask response
        } catch (error) {
            console.log('Fetch Error: flask example post', error);
        }
    }
    
    // Attach the function to the click event
    window.addEventListener("click", handleClick);
</script>
'''))

In [ ]:
# Disable the mouse click handler
display(Javascript("window.removeEventListener('click', handleClick);"))  